In [3]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [10]:
url = "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv"

In [11]:
data = pd.read_csv(url,sep=";")

In [14]:
train,test = train_test_split(data,test_size=0.25, random_state = 42)

In [15]:
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()


test_x = test.drop(['quality'],axis=1).values
test_y = test[['quality']].values.ravel()


train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature = infer_signature(train_x,train_y)

In [28]:
## ANN 

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [   
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
        
    )

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate = params['lr'],momentum=params['momentum']
    ),
    loss = 'mean_squared_error',
    metrics = [keras.metrics.RootMeanSquaredError()]
    )

    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                epochs=epochs,batch_size=16)
        
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse = eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metric('eval_rmse',eval_rmse)

        mlflow.tensorflow.log_model(model,'model',signature=signature)

        return{'loss':eval_rmse,'status':STATUS_OK,'model':model}



In [29]:
def objective(params):
    result = train_model(
        params,
        epochs = 3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y
    )
    return result

In [30]:
space = {
    'lr':hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
    'momentum':hp.uniform('momentum',0.0,1.0)
}

In [31]:
mlflow.set_experiment('/wine-quality')
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo=tpe.suggest,
        max_evals = 4,
        trials = trials 
    )

    best_run = sorted(trials.results,key=lambda x: x['loss'])[0]

    mlflow.log_params(best)
    mlflow.log_metric('eval_rmse',best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'],'model',signature = signature)

    print(f'Best Parameters: {best}')
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

  1/184 ━━━━━━━━━━━━━━━━━━━━ 1:58 649ms/step - loss: 34.2723 - root_mean_squared_error: 5.8543
 20/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.9402 - root_mean_squared_error: 5.3693    
 37/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 24.7962 - root_mean_squared_error: 4.9508
 54/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21.7317 - root_mean_squared_error: 4.6132
 67/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.9139 - root_mean_squared_error: 4.4002
 83/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18.1185 - root_mean_squared_error: 4.1794
 98/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.7675 - root_mean_squared_error: 4.0062
116/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15.4446 - root_mean_squared_error: 3.8300
135/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.3097 - root_mean_squared_error: 3.6733
153/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13.4274 - root_mean_squared_error: 3.5481
171/184 ━━━━━━━━━━━━━━━━━━━